## Homework

In [1]:
#!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])
df1 = df.copy(deep=True)

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [4]:
df

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36.0,7.0,2.0,15.0,33.0,1.0,12.0,75.0,10.0,15318960.0
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,0.0,0.0,13.0,1.0,0.0,6.0,1.0,6080000.0
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13.0,6.0,1.0,8.0,18.0,0.0,1.0,52.0,0.0,17000000.0
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,3.0,0.0,2.0,8.0,2.0,990000.0
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3.0,1.0,0.0,5.0,8.0,0.0,1.0,34.0,5.0,7900000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8011,8011,2014-03-11,67,46.0,2.0,5.0,1.0,1961.0,3.0,8.0,...,5.0,2.0,0.0,8.0,14.0,0.0,1.0,45.0,6.0,3000000.0
8012,8012,2013-06-05,63,36.0,8.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,7.0,1.0,0.0,12.0,1.0,5835375.0
8013,8013,2012-08-27,51,50.0,8.0,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,1.0,9.0,1.0,0.0,16.0,0.0,1000000.0
8014,8014,2014-12-09,69,33.0,12.0,16.0,4.0,2013.0,2.0,12.0,...,15.0,4.0,1.0,10.0,9.0,0.0,1.0,61.0,8.0,16499110.0


Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

In [5]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [6]:
df.isna().sum().sum()

68482

**Много пропусков -- нехорошо**

In [7]:
# Вставим медиану в числовые пропуски и самое частое значение в категориальные
df = df[[*num_columns, *cat_columns]].apply(lambda x: x.fillna(x.median()) if x.dtype.kind in 'biufc' else x.fillna(x.mode().iloc[0])) 

In [8]:
df.isna().sum().sum()

0

**Пропусков не осталось -- красота!**

In [9]:
df[cat_columns].nunique()

product_type                   2
material                       5
state                          4
sub_area                     143
culture_objects_top_25         2
thermal_power_plant_raion      2
incineration_raion             2
oil_chemistry_raion            2
radiation_raion                2
railroad_terminal_raion        2
big_market_raion               2
nuclear_reactor_raion          2
detention_facility_raion       2
ID_metro                     209
ID_railroad_station_walk     125
ID_railroad_station_avto     127
water_1line                    2
ID_big_road1                  47
big_road1_1line                2
ID_big_road2                  57
railroad_1line                 2
ID_railroad_terminal           8
ID_bus_terminal               14
ecology                        5
dtype: int64

**Большинство признаков имеет всего два значения, они отлично подойдут для OHE, а признак sub_area закодируем как категориальный**

In [10]:
df[cat_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8016 entries, 0 to 8015
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_type               8016 non-null   object 
 1   material                   8016 non-null   float64
 2   state                      8016 non-null   float64
 3   sub_area                   8016 non-null   object 
 4   culture_objects_top_25     8016 non-null   object 
 5   thermal_power_plant_raion  8016 non-null   object 
 6   incineration_raion         8016 non-null   object 
 7   oil_chemistry_raion        8016 non-null   object 
 8   radiation_raion            8016 non-null   object 
 9   railroad_terminal_raion    8016 non-null   object 
 10  big_market_raion           8016 non-null   object 
 11  nuclear_reactor_raion      8016 non-null   object 
 12  detention_facility_raion   8016 non-null   object 
 13  ID_metro                   8016 non-null   int64

In [11]:
df.select_dtypes(include=['object'])

,product_type,sub_area,culture_objects_top_25,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,water_1line,big_road1_1line,railroad_1line,ecology
0,OwnerOccupier,Nagornoe,no,no,no,no,no,no,no,no,no,no,no,no,poor
1,OwnerOccupier,Poselenie Sosenskoe,no,no,no,no,no,no,yes,no,no,no,no,no,no data
2,Investment,Krylatskoe,no,no,no,no,no,no,no,no,yes,no,no,no,good
3,Investment,Matushkino,no,no,no,no,no,no,no,no,no,no,no,no,no data
4,Investment,Gol'janovo,no,no,no,no,yes,no,no,no,no,yes,no,no,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8011,Investment,Vostochnoe Izmajlovo,no,no,no,no,yes,no,no,no,no,no,no,no,good
8012,OwnerOccupier,Poselenie Voskresenskoe,no,no,no,no,no,no,no,no,no,yes,no,no,no data
8013,Investment,Juzhnoe Butovo,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
8014,Investment,Juzhnoe Tushino,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory


In [12]:
for column in df.select_dtypes(include=['object']).columns:
    if df[column].nunique() < 5:
        print(df[column])
        df = pd.concat([df.drop([column], axis=1), pd.get_dummies(df[column], prefix=column)], axis=1)
    else:
        df[column] = df[column].astype('category').cat.rename_categories([i for i in range(df[column].nunique())]).astype('int')

0       OwnerOccupier
1       OwnerOccupier
2          Investment
3          Investment
4          Investment
            ...      
8011       Investment
8012    OwnerOccupier
8013       Investment
8014       Investment
8015       Investment
Name: product_type, Length: 8016, dtype: object
0       no
1       no
2       no
3       no
4       no
        ..
8011    no
8012    no
8013    no
8014    no
8015    no
Name: culture_objects_top_25, Length: 8016, dtype: object
0       no
1       no
2       no
3       no
4       no
        ..
8011    no
8012    no
8013    no
8014    no
8015    no
Name: thermal_power_plant_raion, Length: 8016, dtype: object
0       no
1       no
2       no
3       no
4       no
        ..
8011    no
8012    no
8013    no
8014    no
8015    no
Name: incineration_raion, Length: 8016, dtype: object
0       no
1       no
2       no
3       no
4       no
        ..
8011    no
8012    no
8013    no
8014    no
8015    no
Name: oil_chemistry_raion, Length: 8016, dtype: objec

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X, y = df.drop('price', axis=1), df.price

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [16]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [17]:
dtree = DecisionTreeRegressor()
dtree.fit(X_train, y_train)

DecisionTreeRegressor()

Проверьте качество на отложенной выборке.

In [18]:
print("RMSE -- ", mean_squared_error(dtree.predict(X_test), y_test) ** 0.5)
print("MAE -- ", mean_absolute_error(dtree.predict(X_test), y_test))
print("R2 -- ", r2_score(dtree.predict(X_test), y_test))

RMSE --  3584788.708838755
MAE --  1996682.5386533665
R2 --  0.324308172797967


### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [19]:
month_year = (df1.timestamp.dt.month + df1.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df1.timestamp.dt.weekofyear + df1.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/var/folders/h6/_m04r6f57yx4m8bmpscxtckw0000gn/T/ipykernel_79756/548278061.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  week_year = (df1.timestamp.dt.weekofyear + df1.timestamp.dt.year * 100)


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [20]:
import numpy as np

In [21]:
df['month'] = df1.timestamp.dt.month
df['day_of_week'] = df1.timestamp.dt.dayofweek
df['floor_ratio'] = df["floor"] / df["max_floor"]
df['kitchen_ratio'] = df["kitch_sq"] / df["full_sq"]
df[["month", 'day_of_week', 'floor_ratio', 'kitchen_ratio']] = (
    df[["month", 'day_of_week', 'floor_ratio', 'kitchen_ratio']].apply(lambda x: x.fillna(x.median()) if x.dtype.kind in 'biufc' else x.fillna(x.mode().iloc[0])))
df = df[np.isfinite(df).all(1)]

In [22]:
df['kitchen_ratio'].isna().sum()

0

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [23]:
X, y = df.drop('price', axis=1), df.price

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [25]:
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

In [26]:
dtreereg = DecisionTreeRegressor()
catbreg = CatBoostRegressor()
rfr = RandomForestRegressor()

In [27]:
dtree.fit(X_train, y_train)
catbreg.fit(X_train, y_train, verbose=False)
rfr.fit(X_train, y_train)

RandomForestRegressor()

**Чтобы у нас не был результат в квадратах валюты, будем смотреть на RMSE (как и указано в задании). Также я взял MAE и R2, чтобы отслеживать поведение модели на нескольких метриках**

In [28]:
print("DecisionTree score:")
print("RMSE -- ", mean_squared_error(dtree.predict(X_test), y_test) ** 0.5)
print("MAE -- ", mean_absolute_error(dtree.predict(X_test), y_test))
print("R2 -- ", r2_score(dtree.predict(X_test), y_test))

DecisionTree score:
RMSE --  3554849.744541353
MAE --  1958024.1932612841
R2 --  0.30846042036367227


In [29]:
print("CatBoost score:")
print("RMSE -- ", mean_squared_error(catbreg.predict(X_test), y_test) ** 0.5)
print("MAE -- ", mean_absolute_error(catbreg.predict(X_test), y_test))
print("R2 -- ", r2_score(catbreg.predict(X_test), y_test))

CatBoost score:
RMSE --  2618455.185768419
MAE --  1440233.9702441043
R2 --  0.46556192037339905


In [30]:
print("RandomForest score:")
print("RMSE -- ", mean_squared_error(rfr.predict(X_test), y_test) ** 0.5)
print("MAE -- ", mean_absolute_error(rfr.predict(X_test), y_test))
print("R2 -- ", r2_score(rfr.predict(X_test), y_test))

RandomForest score:
RMSE --  2626907.826671893
MAE --  1455496.188862047
R2 --  0.3992174198989704


In [31]:
dtreereg_tuned = DecisionTreeRegressor() # {'max_depth': 7, 'min_samples_leaf': 10}
catbreg_tuned = CatBoostRegressor() # {'depth': 4, 'iterations': 500, 'learning_rate': 0.1}
rfr_tuned = RandomForestRegressor() # {'max_depth': 10, 'max_features': 'auto', 'n_estimators': 500}

rfr_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4, 7, 10]
}

catbreg_grid = {
    'depth': [4, 7, 10],
    'learning_rate': [0.01 , 0.1],
    'iterations': [100, 500, 1000]
}

dtreereg_grid = {
    'max_depth' : [4, 7, 10],
    "min_samples_leaf" : [1, 5, 10]
}

gscv_dtreereg = GridSearchCV(estimator = dtreereg_tuned, param_grid = dtreereg_grid, cv = 5, scoring='neg_root_mean_squared_error')
gscv_rfr = GridSearchCV(estimator = rfr_tuned, param_grid = rfr_grid, cv = 5, scoring='neg_root_mean_squared_error')


In [32]:
grid_search_result = catbreg_tuned.grid_search(catbreg_grid,
                                        cv=5,
                                        X=X,
                                        y=y,
                                        plot=True,
                                        verbose=False)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 8277339.5856430	test: 8192004.8283050	best: 8192004.8283050 (0)	total: 6.59ms	remaining: 652ms
1:	learn: 8211399.9874983	test: 8125621.3021804	best: 8125621.3021804 (1)	total: 12.8ms	remaining: 626ms
2:	learn: 8145484.1752575	test: 8058147.5427001	best: 8058147.5427001 (2)	total: 17.2ms	remaining: 557ms
3:	learn: 8082688.8112464	test: 7996740.3325097	best: 7996740.3325097 (3)	total: 21.6ms	remaining: 518ms
4:	learn: 8019256.9359489	test: 7934743.4480404	best: 7934743.4480404 (4)	total: 26ms	remaining: 495ms
5:	learn: 7955920.2025786	test: 7870045.5282011	best: 7870045.5282011 (5)	total: 32.6ms	remaining: 511ms
6:	learn: 7893831.9037911	test: 7807050.1997731	best: 7807050.1997731 (6)	total: 39.4ms	remaining: 523ms
7:	learn: 7830348.5720712	test: 7744323.1014572	best: 7744323.1014572 (7)	total: 44.4ms	remaining: 511ms
8:	learn: 7769656.2172069	test: 7685347.2113795	best: 7685347.2113795 (8)	total: 49.7ms	remaining: 502ms
9:	learn: 7711511.3588708	test: 7626490.4406400	best: 762

In [33]:
grid_search_result['params']


{'depth': 4, 'iterations': 1000, 'learning_rate': 0.1}

In [34]:
gscv_dtreereg.fit(X, y)


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [4, 7, 10],
                         'min_samples_leaf': [1, 5, 10]},
             scoring='neg_root_mean_squared_error')

In [35]:
gscv_dtreereg.best_params_

{'max_depth': 7, 'min_samples_leaf': 10}

In [36]:
gscv_rfr.fit(X, y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [4, 7, 10],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [200, 500]},
             scoring='neg_root_mean_squared_error')

In [37]:
gscv_rfr.best_params_

{'max_depth': 10, 'max_features': 'auto', 'n_estimators': 500}

In [38]:
dtreereg_tuned = DecisionTreeRegressor(**{'max_depth': 7, 'min_samples_leaf': 10}) # {'max_depth': 7, 'min_samples_leaf': 10}
catbreg_tuned = CatBoostRegressor(**{'depth': 4, 'iterations': 500, 'learning_rate': 0.1}) # {'depth': 4, 'iterations': 500, 'learning_rate': 0.1}
rfr_tuned = RandomForestRegressor(**{'max_depth': 10, 'max_features': 'auto', 'n_estimators': 500}) # {'max_depth': 10, 'max_features': 'auto', 'n_estimators': 500}

In [39]:
dtreereg_tuned.fit(X_train, y_train)
catbreg_tuned.fit(X_train, y_train, verbose=False)
rfr_tuned.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, n_estimators=500)

In [40]:
print("Tuned DecisionTree score:")
print("RMSE -- ", mean_squared_error(dtreereg_tuned.predict(X_test), y_test) ** 0.5)
print("MAE -- ", mean_absolute_error(dtreereg_tuned.predict(X_test), y_test))
print("R2 -- ", r2_score(dtreereg_tuned.predict(X_test), y_test))

Tuned DecisionTree score:
RMSE --  2926671.9249909576
MAE --  1710580.0487257107
R2 --  0.3502076681720655


In [41]:
print("Tuned CatBoost score:")
print("RMSE -- ", mean_squared_error(catbreg_tuned.predict(X_test), y_test) ** 0.5)
print("MAE -- ", mean_absolute_error(catbreg_tuned.predict(X_test), y_test))
print("R2 -- ", r2_score(catbreg_tuned.predict(X_test), y_test))

Tuned CatBoost score:
RMSE --  2505250.917515191
MAE --  1437070.8937287922
R2 --  0.51662519365673


In [42]:
print("Tuned RandomForest score:")
print("RMSE -- ", mean_squared_error(rfr_tuned.predict(X_test), y_test) ** 0.5)
print("MAE -- ", mean_absolute_error(rfr_tuned.predict(X_test), y_test))
print("R2 -- ", r2_score(rfr_tuned.predict(X_test), y_test))

Tuned RandomForest score:
RMSE --  2588160.5902109374
MAE --  1473235.5467026336
R2 --  0.4060948042544338


**Перебором небольшого числа параметров нам удалось улучшить почти все метрики на моделях, исключая catboost... что странно, хотя, может быть дефолтные параметры подходят лучше для этой конкретной задачи**

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [43]:
invest_train_X, invest_test_X, invest_train_Y,  invest_test_Y,  = (
    train_test_split(df[df['product_type_Investment'] == 1].drop(columns=["product_type_Investment", "product_type_OwnerOccupier", "price"]), 
                        df.price[df['product_type_Investment'] == 1], test_size=0.2, random_state=42)
    )

owner_train_X, owner_test_X, owner_train_Y,  owner_test_Y,  = (
    train_test_split(df[df['product_type_Investment'] == 0].drop(columns=["product_type_Investment", "product_type_OwnerOccupier", "price"]), 
                        df.price[df['product_type_Investment'] == 0], test_size=0.2, random_state=42)
    )


In [44]:
catbreg_invest = CatBoostRegressor()
catbreg_owner = CatBoostRegressor()

catbreg_invest.fit(invest_train_X, invest_train_Y, verbose=False)
catbreg_owner.fit(owner_train_X, owner_train_Y, verbose=False)

In [45]:
y_pred = (catbreg_invest.predict(X_test) + catbreg_owner.predict(X_test)) / 2

In [46]:
print("Stacking 2xCatBoost score:")
print("RMSE -- ", mean_squared_error(y_pred, y_test) ** 0.5)
print("MAE -- ", mean_absolute_error(y_pred, y_test))
print("R2 -- ", r2_score(y_pred, y_test))

Stacking 2xCatBoost score:
RMSE --  2111391.6074021743
MAE --  1278728.9628431262
R2 --  0.639814930410589


**Супер, стекингом мы значительно улучшили все метрики!**

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [47]:
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [48]:
catbreg3 = CatBoostRegressor()
hubreg = HuberRegressor(max_iter=1000)


In [49]:
catbreg3.fit(invest_train_X, invest_train_Y, verbose=False)
hubreg.fit(sc.fit_transform(invest_train_X), invest_train_Y)

HuberRegressor(max_iter=1000)

In [50]:
for w in np.arange(0.1, 1.1, 0.2):
    y_pred = w * catbreg3.predict(invest_test_X) + (1 - w) * hubreg.predict(sc.transform(invest_test_X))
    print(f"Stacking CatBoost and HubertRegression score with w={w}:")
    print("RMSE -- ", mean_squared_error(y_pred, invest_test_Y) ** 0.5)
    print("MAE -- ", mean_absolute_error(y_pred, invest_test_Y))
    print("R2 -- ", r2_score(y_pred, invest_test_Y), '\n')

Stacking CatBoost and HubertRegression score with w=0.1:
RMSE --  3137050.2233661083
MAE --  1853071.8583671341
R2 --  -0.0403263232032216 

Stacking CatBoost and HubertRegression score with w=0.30000000000000004:
RMSE --  3031684.972079481
MAE --  1803709.8378822524
R2 --  0.060088248778543574 

Stacking CatBoost and HubertRegression score with w=0.5000000000000001:
RMSE --  2951671.74626097
MAE --  1779974.9192480287
R2 --  0.15063776652957372 

Stacking CatBoost and HubertRegression score with w=0.7000000000000001:
RMSE --  2899110.3958196407
MAE --  1778695.4811322202
R2 --  0.22864002559069907 

Stacking CatBoost and HubertRegression score with w=0.9000000000000001:
RMSE --  2875506.6924553053
MAE --  1801480.0931162694
R2 --  0.29295382551586935 



**Видно, что качество начинает расти лишь при уменьшении весов у hubreg'a, что означает, что он не подходит для решения этой задачи и нужно попробовать стакать другие модели**